## ResNet

* 2015년 ILSVRC 대회에서 우승한 모델이며, 이미지 분류 모델이다.
* Imagenet데이터로 학습되었으며, 1000개의 이미지를 분류 할 수 있다.
* 152층까지 신경망을 깊게 쌓은 모델로, 신경망을 깊게 쌓았을 때 흔히 발생하는 gradient vanishing 문제를 해결한 모델로 유명하다.

In [10]:
# 모델 불러오기
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()

Using cache found in /home/kty/.cache/torch/hub/pytorch_vision_v0.10.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## 이미지 다운로드 및 시각화


In [11]:
from PIL import  Image
import urllib

url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
urllib.request.urlretrieve(url, filename)

# 이미지 불러오기
input_image = Image.open(filename)

# 이미지 시각화
input_image.show()

Unable to init server: 연결할 수 없습니다: 연결이 거부됨

(eog:605669): Gtk-WARNING **: 11:44:22.032: cannot open display: 


## 이미지 전처리 및 추론

In [12]:
from torchvision import transforms # pytorch에서 제공하는 이미지 전처리 라이브러리

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize(256), # 이미지 크기 256으로 변경
    transforms.CenterCrop(224), # 중앙에서 이미지 자르기
    transforms.ToTensor(), # 이미지를 tensor 형태로 변경
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # 이미지 정규화
])
input_tensor = preprocess(input_image)

# 모델의 input에 맞게끔 변경 torch.Size([3, 224, 244]) -> torch.Size([1, 3, 224, 224])
input_batch = input_tensor.unsqueeze(0)

# gpu 사용 가능하면 gpu 사용
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

# 추론
with torch.no_grad():
    output = model(input_batch)

# 확률값을 얻기 위해 softmax 함수 사용
probabilities = torch.nn.functional.softmax(output[0], dim=0)

# 1000개의 클래스에 대한 확률값
print(probabilities)

tensor([1.3643e-06, 1.2945e-06, 8.4374e-07, 6.4035e-07, 2.5654e-07, 9.9503e-06,
        1.1300e-06, 1.9166e-05, 3.1551e-04, 2.2732e-06, 1.0570e-06, 1.3912e-06,
        5.9990e-07, 1.0410e-06, 1.9063e-06, 7.8902e-07, 9.5297e-07, 1.1936e-05,
        3.6116e-06, 1.1967e-06, 6.4429e-07, 1.2895e-06, 4.1575e-07, 2.5459e-06,
        2.7906e-07, 1.5405e-06, 3.0079e-06, 5.2704e-06, 1.2884e-06, 5.6977e-05,
        1.8314e-06, 3.1591e-06, 1.1874e-05, 4.2196e-07, 2.3713e-06, 1.0636e-06,
        6.5259e-07, 9.9344e-07, 2.3006e-06, 1.5493e-06, 5.7597e-07, 3.7489e-07,
        2.3794e-07, 3.2044e-07, 2.3544e-06, 1.0402e-06, 5.0869e-06, 1.3132e-06,
        1.1039e-07, 7.9565e-07, 7.7057e-07, 6.4722e-07, 4.8242e-06, 1.6592e-07,
        1.4689e-06, 1.1018e-06, 1.2219e-06, 2.2007e-06, 1.5694e-06, 6.6987e-07,
        1.0445e-05, 6.0927e-07, 2.6044e-07, 1.8439e-07, 7.4018e-07, 6.5772e-07,
        1.8256e-06, 2.6023e-07, 7.3180e-07, 1.1469e-07, 7.6109e-07, 3.7532e-06,
        2.9881e-07, 1.2419e-06, 8.7862e-

## 이미지 추론 결과 보기

In [13]:
# ImageNet class map 다운로드
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2024-03-11 11:44:38--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 10472 (10K) [text/plain]
저장 위치: `imagenet_classes.txt.1'

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    / 0s       

2024-03-11 11:44:38 (41.2 MB/s) - `imagenet_classes.txt.1' 저장함 [10472/10472]



In [14]:
# txt 파일의 클래스 읽기
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# 확률이 높은 5개의 클래스 출력
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Samoyed 0.7907493114471436
Pomeranian 0.08977577090263367
white wolf 0.03610272333025932
keeshond 0.026814226061105728
Arctic fox 0.02278805896639824
